# Reddit 댓글 규칙 위반 여부 분류

## Import Library

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
!kaggle competitions download -c jigsaw-agile-community-rules
!ls -alh
!unzip -l jigsaw-agile-community-rules.zip

In [ ]:
# 새 폴더 만들기 / -p: 중간 경로가 없어도 부모 폴더까지 같이 생성 (이미 있어도 에러 없이 넘어감)
!mkdir -p data/jigsaw_agile

# 압축 풀기 / -q: 조용히 진행, -o(overwrite): 같은 이름의 파일이 있어도 덮어씀
# -d: 풀린 파일을 저장할 목적지 폴더 지정
!unzip -q -o jigsaw-agile-community-rules.zip -d data/jigsaw_agile

# 폴더 내용 보기 / -a: 숨김 파일까지 모두 표시, -l: 자세히 표시, -h: 파일을 KB/MB 단위로 표시
!ls -alh data/jigsaw_agile

## 1. Load Data

In [ ]:
train_df = pd.read_csv('../data/jigsaw_agile/train.csv')
test_df = pd.read_csv('../data/jigsaw_agile/test.csv')

print(test_df.shape)
print(train_df.shape)

In [ ]:
train_df.info()

In [ ]:
train_df.head()

In [ ]:
# 클래스 불균형 확인
train_df["rule_violation"].value_counts()

## 2. Data Preprocessing

In [ ]:
import re

def preprocess_text(text):
    # 소문자로 통일
    text = str(text).lower()

    # url이 포함된 경우 [URL] 토큰으로 치환
    text = re.sub(r'http\S+|www\S+', '[URL]', text)

    # 특수문자 제거
    text = re.sub(r'[^\w\s[\]]', '', text)

    # 공백 제거
    text = ' '.join(text.split())
    return text

In [ ]:
body_processed = train_df["body"].apply(preprocess_text)
rule_processed = train_df["rule"].apply(preprocess_text)

# 타겟 컬럼 분리
X = body_processed + " [RULE] " + rule_processed
y = train_df["rule_violation"]

In [ ]:
# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
print(X_train)

## 모델 생성

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

clf = pipeline("text-classification",
              model="unitary/toxic-bert")

In [ ]:
sample_predictions = []
for text in X_val[:200]:
    result = clf(text, truncation=True)

    # TOXIC로 예측된 경우 1, 아닌 경우 0으로 변환
    label = 1 if result[0]['label'] == 'TOXIC' else 0
    sample_predictions.append(label)

acc = accuracy_score(y_val[:200], sample_predictions)
print(f"Sample Accuracy: {acc}")